# Composition Example

In this example we're going to compose and link multiple scatter plots.

In [1]:
import numpy as np
import pandas as pd
import jscatter

First we generate some dummy grid data such that each subsequent dataset has have the points.

In [2]:
X1, Y1 = np.mgrid[0:8:1, 0:8:1]
X2, Y2 = np.mgrid[0:4:1, 0:4:1]
X3, Y3 = np.mgrid[0:2:1, 0:2:1]

X = [X1, X2, X3]
Y = [Y1, Y2, Y3]

In [3]:
sc1 = jscatter.Scatter(X1.flatten(), Y1.flatten(), color='#dca237', size=10)
sc2 = jscatter.Scatter(X2.flatten(), Y2.flatten(), color='#6fb2e4', size=10)
sc3 = jscatter.Scatter(X3.flatten(), Y3.flatten(), color='#c17da5', size=10)

To define correspondes upon hovering and selecting, jupyter-scatter's `compose` function allows to specify select and hover mapping functions. The job of a matter is to map an incoming hover or select event to map to the local points that need to be selected.

In [4]:
# In this demo, create_mapper() is a factory function that creates a mapping function for the `i`th scatter plot.
def create_mapper(i):
    from functools import reduce
    from math import floor
    from operator import concat
    
    # This is that target or output number of rows
    target_rows = X[i].shape[1]
    
    # The actually mapping function receives two arguments when a hover or select event comes in:
    # 1. `j` the index of the scatter plot that triggered the event
    # 2. `v` the value of the hover or selection event
    def mapper(j, v): 
        if i == j:
            return v
        
        exp = 2 ** abs(i - j)
        
        is_less = i > j
        
        trans_less = lambda idx: floor(idx * 0.5 ** exp)
        trans_more = lambda idx: list(range(idx * int(2 ** exp), (idx + 1) * int(2 ** exp)))
        
        is_list = isinstance(v, list)
        
        if is_list:
            if is_less:
                return list(dict.fromkeys(map(trans_less, v)))
            else:
                print()
                return reduce(concat, map(trans_more, v))
        
        if is_less:
            return trans_less(v)
        else:
            return list(trans_more(v))
        
    return mapper

In [5]:
jscatter.compose(
    [sc1, sc2, sc3],
    select_mappers=[create_mapper(i) for i in range(3)],
    hover_mappers=[create_mapper(i) for i in range(3)]
)

GridBox(children=(HBox(children=(VBox(children=(Button(button_style='primary', icon='arrows', layout=Layout(wi…

### Grid Compositions

You can also compose multiple scatter plots vertically or in a 2D grid through a nested list of scatter plots.

In [6]:
sc4 = jscatter.Scatter(
    np.random.rand(100), np.random.rand(100), color='#469b76', size=10
)

In [7]:
jscatter.compose([[sc1, sc2], [sc3, sc4]], row_height=200)

GridBox(children=(HBox(children=(VBox(children=(Button(button_style='primary', icon='arrows', layout=Layout(wi…